# Sentiment Analysis_WordCloud
### @Author : Woojin Park, Nidhi Bhaskar
### @Copyright : 2020, Neolth NSF grant NLP project 
### @Email : woojinpa@andrew.cmu.edu , nidhibha@andrew.cmu.edu
### @Status : In-Progress

In [32]:
### Import Relevant Libraries
import os
import pandas as pd
import numpy as np
import collections
import datetime as dt
import requests
import json
import re
import time

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
from scipy.stats import norm

import string
import re
import nltk
from nltk.util import ngrams
from nltk import pos_tag,word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [33]:
### Build a get_date function to convert date format
#### Build a data_creation function to read json data into pandas dataframe

def get_date(created):
    return dt.datetime.fromtimestamp(created)

def data_creation(subreddit) : 
    with open('submissions_'+subreddit+'.json') as f:
        data = json.loads("[" + 
            f.read().replace("}\n{", "},\n{") + 
        "]")
    data =pd.DataFrame(data)
    reddit_data = data[['author','over_18','title','selftext','num_comments', 'score', 'full_link','created_utc']]
    reddit_data = reddit_data.dropna()
    _timestamp = reddit_data["created_utc"].apply(get_date)
    reddit_data = reddit_data.assign(timestamp = _timestamp)
    reddit_data['over_18'] = reddit_data['over_18'].astype('str')
    reddit_data['subreddit']= subreddit
    reddit_data['title_with_selftext']= reddit_data['title'] + reddit_data['selftext']
    # Do one more extra cleaning : keep updating
    reddit_data=reddit_data[~reddit_data['title_with_selftext'].isin([ '[removed]', '[deleted]',''])]
    subreddit = reddit_data
    return subreddit

def empty_words_clean(text):
    text = text.replace('[removed]','')
    text= text.replace('[deleted]','')
    text= text.replace('\n','')
    return (text)

In [34]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [35]:
### Dataframing 4 subreddit Datasets
SuicideWatch_df = data_creation('SuicideWatch')
depressed_df = data_creation('depressed')
happy_df = data_creation('happy')
selfimprovement_df = data_creation('selfimprovement')

### Concat all 4 dataframes into one merged file
all_subreddit_df = pd.concat([SuicideWatch_df,depressed_df,happy_df,selfimprovement_df])
all_subreddit_df.head(2)

author over_18                                              title  \
0    DespressoCafe   False  I don't know where to go or what to do. I can'...   
1  LifeisCrumbling   False                   I'm having an existencial crisis   

                                            selftext  num_comments  score  \
0  Let's make it quick. I'm almost 20. I've been ...             5      1   
1  If I only helped people either as a defense me...             1      1   

                                           full_link  created_utc  \
0  https://www.reddit.com/r/SuicideWatch/comments...   1579669252   
1  https://www.reddit.com/r/SuicideWatch/comments...   1579669256   

            timestamp     subreddit  \
0 2020-01-22 00:00:52  SuicideWatch   
1 2020-01-22 00:00:56  SuicideWatch   

                                 title_with_selftext  
0  I don't know where to go or what to do. I can'...  
1  I'm having an existencial crisisIf I only help...

In [36]:
happy_df.head(3)

author over_18                               title  \
0  francienolan32   False                      I have a crush   
1  alisindianhair   False  ombre hair brazilian hair straight   
2       trex88007   False                Interview in an hour   

                                            selftext  num_comments  score  \
0                                          [removed]             0      1   
1                                          [removed]             0      0   
2  Oh reddit,\nI need this! 11.5 months unemploye...             4      5   

                                           full_link  created_utc  \
0  https://www.reddit.com/r/happy/comments/6claus...   1495429590   
1  https://www.reddit.com/r/happy/comments/6clkfb...   1495433737   
2  https://www.reddit.com/r/happy/comments/6cmrwk...   1495453931   

            timestamp subreddit  \
0 2017-05-22 01:06:30     happy   
1 2017-05-22 02:15:37     happy   
2 2017-05-22 07:52:11     happy   

                                 title_with_selftext  
0                            I have a crush[removed]  
1        ombre hair brazilian hair straight[removed]  
2  Interview in an hourOh reddit,\nI need this! 1...

In [37]:
happy_df["title_with_selftext_new"] = happy_df["title_with_selftext"].apply(lambda x: empty_words_clean(x))
happy_df.head(3)

author over_18                               title  \
0  francienolan32   False                      I have a crush   
1  alisindianhair   False  ombre hair brazilian hair straight   
2       trex88007   False                Interview in an hour   

                                            selftext  num_comments  score  \
0                                          [removed]             0      1   
1                                          [removed]             0      0   
2  Oh reddit,\nI need this! 11.5 months unemploye...             4      5   

                                           full_link  created_utc  \
0  https://www.reddit.com/r/happy/comments/6claus...   1495429590   
1  https://www.reddit.com/r/happy/comments/6clkfb...   1495433737   
2  https://www.reddit.com/r/happy/comments/6cmrwk...   1495453931   

            timestamp subreddit  \
0 2017-05-22 01:06:30     happy   
1 2017-05-22 02:15:37     happy   
2 2017-05-22 07:52:11     happy   

                                 title_with_selftext  \
0                            I have a crush[removed]   
1        ombre hair brazilian hair straight[removed]   
2  Interview in an hourOh reddit,\nI need this! 1...   

                             title_with_selftext_new  
0                                     I have a crush  
1                 ombre hair brazilian hair straight  
2  Interview in an hourOh reddit,I need this! 11....

In [38]:
happy_df["title_with_selftext_new"].head(3).tolist()

['I have a crush',
 'ombre hair brazilian hair straight',
 "Interview in an hourOh reddit,I need this! 11.5 months unemployed, 6+ months active searching-- 3rd interview. I can do this. I've got this. "]

# Sentiment Analysis 
## -Word Cloud 
### Word Cloud on most frequent Words/Nouns/Adjectives for each subreddit
-  Spot the different topics/terms for each subreddit 

1.Suicide Watch

In [6]:
## All Words

In [8]:
## Nouns

In [9]:
## Adjectives